### PA #1

In [187]:
import networkx as nx
from time import time

### Bellman-Ford

In [131]:
# adj = [
#     [1,2,2],
#     [1,3,4],
#     [2,3,1],
#     [2,4,2],
#     [3,5,4],
#     [4,5,2]
# ]

# adj = [
#     [1,2,2],
#     [1,3,4],
#     [2,3,1],
#     [2,4,2],
#     [3,5,4],
#     [4,5,2],
#     [5,6,-2],
#     [6,4,-1]
# ]

# adj = [
#     [1, 2, 1], 
#     [1, 3, 4], 
#     [2, 4, 2], 
#     [3, 4, 3], 
#     [4, 1, -4]
# ]

adj = [
    [1, 2, 1], 
    [1, 3, 4], 
    [2, 4, 2], 
    [3, 4, 3], 
    [4, 1, -2]
]

In [132]:
def initialize_graph(adj):

    g_in = {}
    nodes = set()

    for (u,v,c) in adj:
        nodes.add(u)
        nodes.add(v)
        if v not in g_in:
            g_in[v] = []
        g_in[v].append((u,c))

    nodes = list(nodes)
    nodes_to_ix = {n:i for i,n in enumerate(nodes)}
    
    return nodes, g_in, nodes_to_ix

nodes, g_in, nodes_to_ix = initialize_graph(adj)

print(nodes)
print('')
for k,v in g_in.items():
    print(k,':', v)

[1, 2, 3, 4]

2 : [(1, 1)]
3 : [(1, 4)]
4 : [(2, 2), (3, 3)]
1 : [(4, -2)]


In [133]:
def bellman_ford(s, nodes, g_in, nodes_to_ix):
    
    A = [[float('inf'),]*len(nodes)]
    A[0][nodes_to_ix[s]] = 0
    neg_cycle = False

    for i in range(1,len(nodes)+1):
        a_i = []
        for i_n,n in enumerate(nodes): 
            l_prev = A[i-1][i_n]
            if n not in g_in:
                l_new = float('inf')
            else:
                l_new = min([A[i-1][nodes_to_ix[w]]+e_wv for w,e_wv in g_in[n]])
            a_i.append(min(l_prev, l_new))

        # early stop if there are no path updates
        if a_i==A[-1]:
            break
        else:
            A.append(a_i)

        # detect negative cycles: at i=|V| distances decreased
        if i==len(nodes):
            for l_prev, l in zip(A[i-1],a_i):
                if l<l_prev:
                    neg_cycle = True
    
    if not neg_cycle:
        return A[-1]
    else:
        return False
    
nodes, g_in, nodes_to_ix = initialize_graph(adj)            
res = bellman_ford(1, nodes, g_in, nodes_to_ix)
print(res)

[0, 1, 4, 3]


In [135]:
# reweighting
# adj = [
#     [1,2,-2],
#     [2,3,-1],
#     [3,1,4],
#     [3,4,2],
#     [3,5,-3],
#     [6,4,1],
#     [6,5,-4]
# ]

def reweight(adj):
    
    # initialize original graph g
    nodes, g_in, nodes_to_ix = initialize_graph(adj)
    # create graph g_1 by adding new vertex 0
    adj_rw = [[0,n,0] for n in nodes]
    adj_rw = adj_rw+adj
    # run bellman_ford shortest paths on g_1 form vertex 0
    nodes_rw, g_in_rw, nodes_to_ix_rw = initialize_graph(adj_rw)
    res = bellman_ford(0, nodes_rw, g_in_rw, nodes_to_ix_rw)
    if res:
        dist_rw = {n:d for (n,d) in zip(nodes,res[1:])}
        # compute new weights on original graph g
        adj_rw = []
        for (u,v,c) in adj:
            c_1 = c + dist_rw[u] - dist_rw[v]
            adj_rw.append([u,v,c_1]) 
        return adj_rw, dist_rw
    else:
        return False

res = reweight(adj)
if res:
    adj_rw, dist_rw = res[0], res[1]

In [136]:
res

([[1, 2, 0], [1, 3, 2], [2, 4, 1], [3, 4, 3], [4, 1, 0]],
 {1: -2, 2: -1, 3: 0, 4: 0})

### Dijkstra

In [137]:
def dijkstra(adj,s=1):
    
    # initialize directed graph
    G = nx.DiGraph()
    for e in adj:
        G.add_edge(e[0],e[1],weight=e[2])
    distance,path = nx.single_source_dijkstra(G,source=s, weight='weight')
    
    return distance,path
        
distance,path = dijkstra(adj_rw,s=1)
print(distance)
print(path)

{1: 0, 2: 0, 4: 1, 3: 2}
{1: [1], 2: [1, 2], 3: [1, 3], 4: [1, 2, 4]}


### Single Source Johnson's Algorithm

In [143]:
adj = [
    [1,2,-2],
    [2,3,-1],
    [3,1,4],
    [3,4,2],
    [3,5,-3],
    [6,4,1],
    [6,5,-4]
]

nodes, _, _ = initialize_graph(adj)
res = reweight(adj)
if res:
    adj_rw, dist_rw = res[0],res[1]
    distance,path = dijkstra(adj_rw,s=1)

    shortest_paths = {}
    for n in nodes:
        if n in distance:
            shortest_paths[n] = distance[n]-dist_rw[s]+dist_rw[n]
        else:
            shortest_paths[n] = float('inf')
else:
    shortest_paths = False
print(shortest_paths)

{1: 0, 2: -2, 3: -3, 4: -1, 5: -6, 6: inf}


### All Pairs Johnson's Algorithm

In [192]:
# assignment data
adj = []
with open('g3.txt') as f:
    for i,l in enumerate(f):
        l = tuple(map(int, l.split()))
        if i!=0:
            adj.append(l)

start = time()
shortest_path = float('inf')
nodes, _, _ = initialize_graph(adj)
res = reweight(adj)
if res:
    adj_rw, dist_rw = res[0],res[1]
    
    for v in nodes:
        distance,path = dijkstra(adj_rw,s=v)
        # correct shortest paths using reweighted distances
        shortest_paths = {}
        for n in nodes:
            if n in distance:
                shortest_paths[n] = distance[n]-dist_rw[v]+dist_rw[n]
            else:
                shortest_paths[n] = float('inf')
        min_path = min(shortest_paths.values())
        if min_path < shortest_path:
            shortest_path = min_path
else:
    shortest_path = False
    
print(shortest_path)
print('elapsed = ', time()-start)

-19
elapsed =  84.4589900970459


### Compare with All Pairs Bellman-Ford Algorithm

In [193]:
start = time()
shortest_path = float('inf')

nodes, g_in, nodes_to_ix = initialize_graph(adj)  
for v in nodes:
    res = bellman_ford(v, nodes, g_in, nodes_to_ix)
    min_path = min(res)
    if min_path < shortest_path:
            shortest_path = min_path

print(shortest_path)
print('elapsed = ', time()-start)

-19
elapsed =  225.40319895744324
